In [ ]:
import warnings
warnings.filterwarnings("ignore")
import joblib
import gc
import numpy as np
import pandas as pd
import _pickle as cpickle
import pickle
from datetime import datetime
import time
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

In [ ]:
# Loading train and test features 
!pip3 install pickle5
import pickle5 as pickle

with open("/content/drive/MyDrive/data_train_final.pkl", "rb") as fh:
  data_train = pickle.load(fh)
with open("/content/drive/MyDrive/data_test_final.pkl", "rb") as fh:
  data_test = pickle.load(fh)  

     |████████████████████████████████| 133kB 4.3MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219248 sha256=d6b862f23202c89dc31b85653cd4475c420914d952c168a5fd4dd81e43b17e43
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


In [ ]:
# Dropping the unimportant columns from both test and train 
data_train.drop(['user_id','min_number_of_orders'],axis=1,inplace=True)
data_test.drop(['user_id','min_number_of_orders'],axis=1,inplace=True)

# replace Nan values with mean values
data_train.fillna(data_train.mean(), inplace=True)
data_test.fillna(data_test.mean(), inplace=True)

In [ ]:
data_train.head()

,product_id,times_brought,uxp_total_reordered,uxp_first_order_num,uxp_avg_cart_pos,uxp_min_cart_pos,uxp_max_cart_pos,uxp_median_cart_pos,uxp_last_order_num,uxp_avg_dow,uxp_median_dow,uxp_avg_hour_of_the_day,uxp_median_hour_of_the_day,uxp_order_rate,uxp_sum_cart_pos,uxp_min_order_days,uxp_max_order_days,uxp_median_order_days,uxp_reorder_ratio,order_streak,max_number_of_orders,mean_number_of_orders,max_day_of_week,max_order_hour_of_day,sum_days_since_prior_order,mean_days_since_prior_order,reordered_ratio,total_num_orders,user_total_prods,user_unique_prods,user_average_basket,user_order_starts_at,avg_no_prds_each_purchase,median_no_prds_each_purchase,min_no_prds_each_purchase,max_no_prds_each_purchase,times_purchased,prod_tot_reorders,prod_reordered_ratio,avg_cart_position,median_cart_position,sum_cart_position,min_cart_position,max_cart_position,std_cart_position,aisle_reorder_ratio,department_reorder_ratio,order_id,reordered
0,196,10,9,1,1.400391,1,4,1.0,10,2.500000,2.5,10.296875,8.5,1.000000,14,0.0,30.0,19.5,0.899902,5.0,10,5,4,4,1150.0,19,0.694824,343,59,18,0.171997,431534,5.898438,5.5,4,9,35791,27791,0.776367,3.722656,2.0,133206,1,60,4.109375,0.638672,0.653320,1187899,1.0
1,10258,9,8,2,3.333984,2,6,3.0,10,2.554688,3.0,10.554688,9.0,0.899902,30,0.0,30.0,20.0,0.888672,5.0,10,5,4,4,1150.0,19,0.694824,343,59,18,0.171997,431534,5.898438,5.5,4,9,1946,1389,0.713867,4.277344,3.0,8324,1,29,3.568359,0.519043,0.574219,1187899,1.0
2,10326,1,0,5,5.000000,5,5,5.0,5,4.000000,4.0,15.000000,15.0,0.099976,5,28.0,28.0,28.0,0.000000,NaN,10,5,4,4,1150.0,19,0.694824,343,59,18,0.171997,431534,5.898438,5.5,4,9,5526,3603,0.651855,4.191406,3.0,23160,1,37,3.611328,0.718262,0.649902,1187899,0.0
3,12427,10,9,1,3.300781,1,9,2.5,10,2.500000,2.5,10.296875,8.5,1.000000,33,0.0,30.0,19.5,0.899902,5.0,10,5,4,4,1150.0,19,0.694824,343,59,18,0.171997,431534,5.898438,5.5,4,9,6476,4797,0.740723,4.761719,3.0,30826,1,61,4.781250,0.591797,0.574219,1187899,0.0
4,13032,3,2,2,6.332031,5,8,6.0,10,2.666016,3.0,8.000000,8.0,0.300049,19,15.0,30.0,20.0,0.666504,1.0,10,5,4,4,1150.0,19,0.694824,343,59,18,0.171997,431534,5.898438,5.5,4,9,3751,2465,0.657227,5.621094,4.0,21091,1,66,5.343750,0.571777,0.561035,1187899,1.0


<h3>Function 1 - 
Pipeline -> Raw Input -> Output Predictions</h3>

In [ ]:
import pdb
def fun1(order_id,dataset_type):
  """
  This function accepts an order id and the type of dataset the order belongs to.
  It returns the list of product ids that the user would purchase again.
  """
  # Load trained catboost model
  model = pickle.load(open("/content/drive/MyDrive/Colab Notebooks/catboost_model", 'rb'))
  data = pd.DataFrame()

  if dataset_type == 'train':
    order_ids = data_train['order_id'].unique()
    if order_id not in order_ids:
      print("Order id is not found in train data")
      return
    else:
      # get all the rows corresponding to the passed order id   
      data = data_train[data_train['order_id'] == order_id]
      
  elif dataset_type == 'test':
    order_ids = data_test['order_id'].unique()
    if order_id not in order_ids:
      print("Order id is not found in test data")
      return
    else:
      # get all the rows corresponding to the passed order id   
      data = data_test[data_test['order_id'] == order_id]

  # Get the preditions for the above filtered data
  # While getting the predictions remove the product id and order id
  predictions = (model.predict_proba(data.drop(['product_id','order_id','reordered'],axis=1))[:, 1] >= 0.21).astype('int')

  # add the predictions as a new column to the above filtered data
  data['predictions'] = predictions

  # filter out all the products whereever predictions = 1(meaning the product has to be reordered)
  products = data[data['predictions'] == 1]['product_id']

  return products.values.tolist() 


In [ ]:
order_id = int(input('Enter the order id: '))
dataset_type = str(input('Enter the dataset type to which the order belongs to: '))
start = datetime.now()
prod_list = fun1(order_id,dataset_type)
print('Products to be reordered are',prod_list)
print('Time taken to execute : ',datetime.now()-start)

Enter the order id: 111
Enter the dataset type to which the order belongs to: train
Order id is not found in train data
Products to be reordered are None
Time taken to execute :  0:00:00.143608


In [ ]:
order_id = int(input('Enter the order id: '))
dataset_type = str(input('Enter the dataset type to which the order belongs to: '))
start = datetime.now()
prod_list = fun1(order_id,dataset_type)
print('Products to be reordered are',prod_list)
print('Time taken to execute : ',datetime.now()-start)

Enter the order id: 1716008
Enter the dataset type to which the order belongs to: train
Products to be reordered are [3896, 21137, 22035, 27845, 38739, 39160, 43352]
Time taken to execute :  0:00:01.496003


In [ ]:
order_id = int(input('Enter the order id: '))
dataset_type = str(input('Enter the dataset type to which the order belongs to: '))
start = datetime.now()
prod_list = fun1(order_id,dataset_type)
print('Products to be reordered are',prod_list)
print('Time taken to execute : ',datetime.now()-start)

Enter the order id: 2821651
Enter the dataset type to which the order belongs to: test
Products to be reordered are [3397, 11068, 13176, 20899, 20995, 22035, 25133, 27344, 27845, 27966, 33716, 33754, 33787, 36011, 39180, 44632, 47209, 48086]
Time taken to execute :  0:00:00.131055


<h3>Function 2 - Pipeline -> (X,y) -> Metric

In [ ]:
def fun2(order_id):
  """
  This function makes use of trained data only. It takes one order id 
  as input and calculates the F1 score for the passed order id.
  """
  # Load trained catboost model
  model = pickle.load(open("/content/drive/MyDrive/Colab Notebooks/catboost_model", 'rb'))
  data = pd.DataFrame()

  order_ids = data_train['order_id'].unique()
  if order_id not in order_ids:
    print("Order id is not found in train data")
    return
  else:
    # get all the rows corresponding to the passed order id   
    data = data_train[data_train['order_id'] == order_id]
      
  # Get the preditions for the above filtered data
  # While getting the predictions remove the product id and order id
  predictions = (model.predict_proba(data.drop(['product_id','order_id','reordered'],axis=1))[:, 1] >= 0.21).astype('int')

  # add the predictions as a new column to the above filtered data
  data['predictions'] = predictions

  data = data[['reordered','predictions']]
  print("The F1_score for order id - {} is {}".format(order_id,
                                        f1_score(data['reordered'],data['predictions'])))


In [ ]:
order_id = int(input('Enter the order id: '))
start = datetime.now()
prod_list = fun2(order_id)
print('Time taken to execute : ',datetime.now()-start)

Enter the order id: 1187899
The F1_score for order id - 1187899 is 0.8000000000000002
Time taken to execute :  0:00:00.173351


In [ ]:
order_id = int(input('Enter the order id: '))
start = datetime.now()
prod_list = fun2(order_id)
print('Time taken to execute : ',datetime.now()-start)

Enter the order id: 2821651
Order id is not found in train data
Time taken to execute :  0:00:00.155722


In [ ]:
order_id = int(input('Enter the order id: '))
start = datetime.now()
prod_list = fun2(order_id)
print('Time taken to execute : ',datetime.now()-start)

Enter the order id: 1716008
The F1_score for order id - 1716008 is 0.4615384615384615
Time taken to execute :  0:00:00.175228


In [ ]:
order_id = int(input('Enter the order id: '))
start = datetime.now()
prod_list = fun2(order_id)
print('Time taken to execute : ',datetime.now()-start)

Enter the order id: 1
The F1_score for order id - 1 is 0.5714285714285715
Time taken to execute :  0:00:00.179621
